In [1]:
import sys
sys.path.append('../')


In [3]:
import spacy
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from math import floor
from nlp_helper import read_glove_vecs, convert_to_one_hot, load_corpus,clean_url_replace_hashtag

In [4]:
MAX_LEN = 80

In [5]:
nlp=spacy.load('en')

In [6]:
neg_data, pos_data = load_corpus('../../data/twitter/6s8c/neg_14202_train.cleaned.txt'), load_corpus('../../data/twitter/6s8c/pos_14202_train.cleaned.txt')

In [51]:

X_train_pos = np.array(pos_data).reshape(len(pos_data),)# shape of (m,)
X_train_neg = np.array(neg_data).reshape(len(neg_data),)
del neg_data, pos_data

In [7]:
np.random.seed(1)
X_train_pos_indices_ori = np.load("../../data/twitter/X_train_pos_indices_14202.npy")
np.random.shuffle(X_train_pos_indices_ori)
Y_train_pos_oh_ori =  convert_to_one_hot(np.array([ 1 for i in X_train_pos_indices_ori]),2)
# X_train_pos_indices = sentences_to_indices(X_train_pos, w_to_ix, MAX_LEN)
# np.save("../../data/X_train_pos_indices_14202.npy",X_train_pos_indices)

In [8]:
X_train_pos_indices, X_validate_pos_indices = X_train_pos_indices_ori[:1000], X_train_pos_indices_ori[1000:]

In [9]:
Y_train_pos = np.array([ 1 for i in X_train_pos_indices])
Y_train_pos_oh  = convert_to_one_hot(Y_train_pos, C = 2)
Y_validate_pos = np.array([ 1 for i in X_validate_pos_indices])
Y_validate_pos_oh = convert_to_one_hot(Y_validate_pos, C = 2)

In [10]:
X_train_neg_indices = np.load("../../data/twitter/X_train_neg_indices_14202.npy")
# X_train_neg_indices = sentences_to_indices(X_train_neg, w_to_ix, MAX_LEN)
# np.save("../../data/X_train_neg_indices_14202.npy",X_train_neg_indices)

In [11]:
X_train_neg_indices, X_validate_neg_indices = X_train_neg_indices[:10000],X_train_neg_indices[10000:]

In [12]:
Y_train_neg = np.array([ 0 for i in X_train_neg_indices])
Y_train_neg_oh= convert_to_one_hot(Y_train_neg, C = 2)
Y_validate_neg = np.array([ 0 for i in X_validate_neg_indices])
Y_validate_neg_oh = convert_to_one_hot(Y_validate_neg, C = 2)

In [13]:
X_train_indices = np.concatenate((X_train_pos_indices, X_train_neg_indices), axis = 0)
Y_train_oh = np.concatenate((Y_train_pos_oh, Y_train_neg_oh), axis = 0)
X_validate_indices = np.concatenate((X_validate_pos_indices, X_validate_neg_indices), axis = 0)
Y_validate_oh = np.concatenate((Y_validate_pos_oh, Y_validate_neg_oh), axis = 0)

In [14]:
np.random.seed(1)
np.random.shuffle(X_train_indices)
np.random.seed(1)
np.random.shuffle(Y_train_oh)

In [182]:
# #加载训练数据
# pos_datas = np.load("../../data/twitter/tech_not/pos_cleaned.npy")
# neg_datas = np.load("../../data/twitter/tech_not/neg_cleaned.npy")

# np.random.seed(1)
# np.random.shuffle(pos_datas)
# np.random.shuffle(neg_datas)

# pos_datas_train, pos_datas_test = pos_datas[:700],pos_datas[700:800]
# neg_datas_train_10, neg_datas_test_10 = neg_datas[:7000],neg_datas[7000:8000]
# neg_datas_train_5, neg_datas_test_5 = neg_datas[13600:17100],neg_datas[17100:20000]
# neg_datas_train_1_2 = neg_datas[20000:20350]
# neg_datas_train_2, neg_datas_test_2 = neg_datas[12000:13400],neg_datas[13400:13600]

In [15]:
w_to_ix, ix_to_w, w_to_vec_map  = read_glove_vecs("../../data/embedding/glove.twitter.27B.50d.txt")

In [5]:
# train1_2 = np.concatenate((pos_datas_train,neg_datas_train_2), axis = 0)

# np.random.shuffle(train1_2)
# X_train_1_2 = train1_2[:,1]
# Y_train_1_2 = np.array(list(map(int, train1_2[:,0])))
# train1_5 = np.concatenate((pos_datas_train,neg_datas_train_5), axis = 0)

# np.random.shuffle(train1_5)
# X_train_1_5 = train1_5[:,1]
# Y_train_1_5 = np.array(list(map(int, train1_5[:,0])))

In [5]:
# pos_datas_50augs = np.tile(pos_datas, (50,1))


In [6]:
# train_auged = np.concatenate((pos_datas_50augs,neg_datas), axis = 0)
# train_
# np.random.shuffle(train_auged )

# X_train, X_test = train_auged[:,1][0:80000], train_auged[:,1][80000:]
# Y_train, Y_test = np.array(list(map(int, train_auged[:,0])))[0:80000], np.array(list(map(int, train_auged[:,0])))[80000:]

In [17]:
w_to_vec_map["cucumber"].shape[0]

50

In [18]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        if word_to_vec_map[word].shape[0]!=50 :
            print("word:", word)
            continue
        emb_matrix[index] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [19]:
# GRADED FUNCTION: Emojify_V2

def twitter_tech_classify_V1(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype ='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation(activation='softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs =sentence_indices ,outputs=X)
    
    
    return model

In [20]:

model = twitter_tech_classify_V1((MAX_LEN,), w_to_vec_map, w_to_ix)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 80, 50)            59675700  
_________________________________________________________________
lstm_1 (LSTM)                (None, 80, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
__________

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.load_weights('weights828.h5')

In [206]:
model.fit(X_train_indices, Y_train_oh, validation_data=[X_validate_indices, Y_validate_oh], epochs = 100, batch_size = 4096, shuffle=True, class_weight = {0:1,1:50})

Train on 11000 samples, validate on 3205 samples
Epoch 1/100
11000/11000 [==============================] - ETA: 1s - loss: 6.6826 - acc: 0.976 - ETA: 0s - loss: 5.4455 - acc: 0.946 - 3s 237us/step - loss: 4.5737 - acc: 0.8944 - val_loss: 2.3631 - val_acc: 0.6343
Epoch 2/100
11000/11000 [==============================] - ETA: 1s - loss: 2.3129 - acc: 0.674 - ETA: 0s - loss: 1.9661 - acc: 0.685 - 3s 236us/step - loss: 1.8434 - acc: 0.7059 - val_loss: 0.8455 - val_acc: 0.7738
Epoch 3/100
11000/11000 [==============================] - ETA: 1s - loss: 1.6780 - acc: 0.824 - ETA: 0s - loss: 1.6550 - acc: 0.814 - 3s 235us/step - loss: 1.6011 - acc: 0.7915 - val_loss: 1.0518 - val_acc: 0.6802
Epoch 4/100
11000/11000 [==============================] - ETA: 1s - loss: 1.4383 - acc: 0.636 - ETA: 0s - loss: 1.4790 - acc: 0.622 - 3s 238us/step - loss: 1.4970 - acc: 0.6168 - val_loss: 1.1885 - val_acc: 0.6324
Epoch 5/100
11000/11000 [==============================] - ETA: 1s - loss: 1.5218 - acc: 0.

Epoch 39/100
11000/11000 [==============================] - ETA: 1s - loss: 0.6380 - acc: 0.932 - ETA: 0s - loss: 0.5862 - acc: 0.933 - 3s 238us/step - loss: 0.5954 - acc: 0.9395 - val_loss: 0.5670 - val_acc: 0.8771
Epoch 40/100
11000/11000 [==============================] - ETA: 1s - loss: 0.6209 - acc: 0.953 - ETA: 0s - loss: 0.5685 - acc: 0.951 - 3s 244us/step - loss: 0.5768 - acc: 0.9504 - val_loss: 0.5949 - val_acc: 0.8705
Epoch 41/100
11000/11000 [==============================] - ETA: 1s - loss: 0.5653 - acc: 0.946 - ETA: 0s - loss: 0.5546 - acc: 0.947 - 3s 245us/step - loss: 0.5656 - acc: 0.9485 - val_loss: 0.5926 - val_acc: 0.8715
Epoch 42/100
11000/11000 [==============================] - ETA: 1s - loss: 0.6450 - acc: 0.952 - ETA: 0s - loss: 0.5994 - acc: 0.950 - 3s 248us/step - loss: 0.5646 - acc: 0.9515 - val_loss: 0.5846 - val_acc: 0.8733
Epoch 43/100
11000/11000 [==============================] - ETA: 1s - loss: 0.5572 - acc: 0.955 - ETA: 0s - loss: 0.5732 - acc: 0.952 - 

Epoch 77/100
11000/11000 [==============================] - ETA: 1s - loss: 0.6521 - acc: 0.946 - ETA: 0s - loss: 0.5775 - acc: 0.946 - 3s 244us/step - loss: 0.5160 - acc: 0.9482 - val_loss: 0.6575 - val_acc: 0.8593
Epoch 78/100
11000/11000 [==============================] - ETA: 1s - loss: 0.4489 - acc: 0.952 - ETA: 0s - loss: 0.4911 - acc: 0.952 - 3s 242us/step - loss: 0.4858 - acc: 0.9532 - val_loss: 0.6159 - val_acc: 0.8677
Epoch 79/100
11000/11000 [==============================] - ETA: 1s - loss: 0.4964 - acc: 0.957 - ETA: 0s - loss: 0.4622 - acc: 0.959 - 3s 240us/step - loss: 0.4757 - acc: 0.9588 - val_loss: 0.5614 - val_acc: 0.8786
Epoch 80/100
11000/11000 [==============================] - ETA: 1s - loss: 0.4920 - acc: 0.962 - ETA: 0s - loss: 0.4900 - acc: 0.961 - 3s 238us/step - loss: 0.4761 - acc: 0.9605 - val_loss: 0.6138 - val_acc: 0.8705
Epoch 81/100
11000/11000 [==============================] - ETA: 1s - loss: 0.4873 - acc: 0.955 - ETA: 0s - loss: 0.4379 - acc: 0.957 - 

In [111]:
loss,accuracy = model.evaluate(X_train_pos_indices_ori,Y_train_pos_oh_ori, batch_size = 128)

1344/1344 [==============================] - 2s 1ms/step


In [112]:
print(loss, accuracy)

0.183562170271 0.94568452381


In [59]:
X = np.array(["AI 5g"])
X = sentences_to_indices(X, w_to_ix, MAX_LEN)
model.predict(X)

array([[ 0.00719968,  0.99280035]], dtype=float32)

In [113]:
ans = model.predict(X_validate_pos_indices)

In [114]:
len(X_validate_pos_indices)

344

In [115]:
count = 0
for i,a in enumerate(ans):
    if a[0]>a[1]:
        
        print(count)
        print(indices_to_sentence(X_validate_pos_indices[i],ix_to_w))
        print('\n')
        count+=1


0
thanks to the virtual reality lab for the incredible training on mass handling in space ! and i practiced handling several pieces of equipment , to include a pound ammonia tank nbd . … 


1
delivering battle winning air power : typhoon jets from continue to degrade and on operations in the middleeast as part of the . . 


2
. ’s deep space exploration program was highlighted at madeinamerica hosted by the . we ’re a proud contributor to , & . learn more about : 


3
the nightshift is about to get a new friend . has partnered with to bring the world ’s first autonomous indoor surveillance drone . 


4
look at those baby planets ! an international team of astronomers used archival radio telescope data & supercomputers to develop a new method for finding very young planets : . 


5
blockchain data now available on google cloud ! a live updating public dataset for smart contract analytics by and cc blockchain 


6
with suppliers in all states from hawaii to maine , the deep space explora

In [29]:
model.save_weights("weights100epoch_978_878_828.h5")


In [63]:
model.save("model_weights_828.h5")

In [62]:
label_file('30000.txt',1,1000)

"HOWDY! Despite the cold... We are in TEXAS folks!! Come warm up at booth 3223 with a latte! Find your #BunnyPin now, before it’s too late! We are Innovation! We are #intelHPC , this is #SC18 ! We are excited to listen to your story!https://twitter.com/intelhpc/status/1062585970606960640 …"

1 :非科技

"#Nuclear energy provides 20% of our nation’s electricity, but plants continue to be shut down. Join us to help preserve this crucial #cleanenergy source. http://Bit.ly/nuclearmattersjoin …pic.twitter.com/ndsIer1X7k"

2 :非科技

"Our Tech Learning Lab is designed to engage with students and educators, sparking conversations that go beyond the classroom to fuel curiosity for #AI and #IoT. #edtech https://intel.ly/2QH0o5K pic.twitter.com/RuO55ZL0zZ"

3 :科技

"In honor of the 100th episode of ""Outrageous Acts of Science,""  we‘ve unlocked a whole library of episodes for you to watch online now. From the absurd to the amazing, these stories will BLOW YOUR MIND! Watch Now --->http://bit.ly/2DF3yEe 

39 :非科技

"Poor dental health may prevent Airmen from completing their job, so the 14th Medical Operations Squadron Dental Flight works consistently to ‘put the bite in the fight’.https://go.usa.gov/xPfMa "

40 :非科技

"RELEASE: Committee Leaders @RepEBJ, @FrankPallone, and @RepRaulGrijalva Announce Series of Climate Hearings over Two Day Period Early Next Yearhttps://democrats-science.house.gov/news/press-releases/committee-leaders-announce-series-climate-hearings-over-two-day-period-early …"

41 :非科技

"Today in the NASA booth (2609) at #SC18, Hoot Thompson will describe a #Cloud capability combining onsite #HPC clusters & private cloud systems that @NASA_NCCS is investigating, 11:40am in our small theater. http://go.nasa.gov/2DB3FRh "

42 :科技

"Secure Chorus hosts its 2nd ‘Thought Leadership Platform‘ focused on data security in the financial services sector http://rlw.co/ahD "

43 :非科技

"Targeting of mitochondrial metabolism in combination with BCL-2 inhibition eradicates #leukemia ste

79 :非科技

"Attention all #SC18 conference goers: We’ve crafted a fun collection of Supercomputing 2018 #Intel #BunnyPeople pins! Collect all 47 variations by visiting our participating partners on the exhibit floor and win the title of #SC18 SUPER Fan!pic.twitter.com/ZBtIji7XCr"

80 :非科技

"5G isn‘t just about making our cellphones better, says @Arm CEO @simonsegars at #WSJDLIVE. It can ""connect billions and billions of devices to the network... and be the oxygen behind AI.""https://on.wsj.com/2DEnHu7 "

81 :科技

"HOMER Energy CEO, Peter Lilienthal, is speaking today at the ECOWAS 2nd annual Sustainable Energy Forum in Dakar, Senegal on standardizing #minigrid modeling, feasibility and financial analyses. #ECOWASSustainableEnergyForumLearn more: https://buff.ly/2DmjwD3 pic.twitter.com/4dem2SRBWi"

82 :非科技

"This summer, six finalists for the 2018 APS LeRoy Apker Award traveled to Washington, D.C. to present their undergraduate research to a panel of judges. Read more on the winners in th

117 :非科技

"In today‘s @NYTimes crossword: 3 letters for ""Big name in #cloud computing."" HINT: The first letter is not . And the second letter is .pic.twitter.com/U4ciFnYkoz"

118 :非科技

"Baglietto, Saxe, and Shoulders ensure grad students thrive - MIT Newshttp://dlvr.it/QrFWNr "

119 :非科技

"The wildfire in Malibu has hit a former nuclear research site, and some activists are worried about radiation in the smoky airhttps://read.bi/2PwLQsL "

120 :非科技

"Congratulations to Canon Hale for his 2nd Place finish in the Annual Congressional APP Challenge. Canon received a certificate from Congressman Bruce Westerman."

121 :非科技

"Nice overview of how the @NASA/@esa/@csa_asc #JWST will improve our understanding of jets & outflows from young stars. Article by @cristinepulliam, featuring Tom Ray, Ewine van Dishoeck, Alberto Noriega-Crespo, & my @eso VLT image of HH212 https://www.nasa.gov/feature/goddard/2018/nasa-s-webb-telescope-will-investigate-cosmic-jets-from-young-stars …"

122 :非科技

"NEW 

158 :非科技

"“It’s a huge consumer issue,” says Richard Lowes at the University of Exeter in the UK. “Yet if you asked the average person in the street about this, they would have no idea what you are talking about.”"

159 :非科技

"Join the FoundationDB Summit in #Seattle Dec. 10 -- the technical conference for companies and developers using #FoundationDB, the recently open-sourced distributed database. Learn more and register at: https://bit.ly/2RL8QB6 pic.twitter.com/CmQV3vOr1D"

160 :非科技

"The Evolving Role of the IoT Edge Network Gateway”, a recent #IDC research paper, discusses how #IoT edge network gateway are incorporating advance features and processing capability to address functional requirement at edge. Download Now  https://bit.ly/2DEUyz2 pic.twitter.com/9Ls9QyWhwm"

161 :科技

"Tech industry: “where are all the senior women?”Also tech industry: “here are 30 amazing women under 30!”"

162 :非科技

"Google’s Alphabet and SoftBank value ‘Uber of trucks’ startup at $6 billion. A payday

...

818 :科技

"Our new scholarship program awards students with more than just a check; mentorship and internship opportunities offer students guides in STEM and beyond! Learn more: https://lockheedmartin.com/scholarship pic.twitter.com/KHFQQRuSR4"

819 :非科技

"Meet the #bugbounty #hackers keeping the #Internet safe: http://bit.ly/2z9siAI  via @mashable #WomenInSTEM"

820 :非科技

"SCIENCE | Much of the current research at the Nanomechanics Laboratory involves red blood cell diseases such as malaria and sickle cell anemia.http://ow.ly/PFWE30mCK1i "

821 :科技

"With mobile data traffic expected to increase dramatically in the foreseeable future, small cell base stations will play an important role in expanding the targeted capacity of wireless networks.  Take the 4 December ""Fundamentals of 5G Small... http://bit.ly/smallcells2018 "

822 :科技

".@USEnergyAssn chief Barry Worthington tells @CanEmbUSA Lines are blurring between #energyefficiency and #energysupplyCan’t have #efficiency without #supp

854 :非科技

"The very nature of your company and its culture may define whether you win or lose in a #digitaltransformation. Check out the characteristics at play. https://ubm.io/2qN54vo  @ckharvey @WestMonroe @FujitsuAmerica #CIO #executivepic.twitter.com/LTcKZe4dQM"

855 :非科技

"Firefighting equipment finds a new home at @atcrockhillsc helping students prepare for emergency management careers #AmericaRecyclesDayhttp://spr.ly/6017E8ZAU "

856 :非科技

"UK Texas Mission within the robotics surgery facility, thanks for the warm welcome! @MethodistHosp @InnovateUK @KTNUK @UKRI_News @UKSINet #TexasDigitalHealth #GlobalExpertMission #USA #Houston #Healthcare #HealtTech #MedTech #Technology #Innovation #mHealth #NHS #AIpic.twitter.com/5ISMsgeFUj"

857 :非科技

"Student wins Congressional app challenge with focus on special needs peers http://bit.ly/2DF4IQ1 pic.twitter.com/JAm9Lm0TJG"

858 :科技

"In the new episode of the ""Smashing Security"" podcast, we ask does your company want to make you a cybor

891 :非科技

"In just six years Microsoft has gone from making accessory hardware to making its own laptops to making workstations like this. Unless you‘re a multimedia professional or have a serious Forza obsession you probably don‘t need it, but it‘s a thing to beholdhttps://www.wired.com/review/microsoft-surface-studio-2/ …"

892 :非科技

"#DYK: 57% of individuals would use a smartphone if searching online for a lawyer? Here are 8 more reasons why legal services firms could consider an app! #appdevelopment #lawfirm #mobileapp https://hubs.ly/H0fvPf-0 pic.twitter.com/LKqALlwaM7"

893 :非科技

"Donald Tusk with a playful suggestion that Brexit could be cancelled: ""The EU is prepared for a final deal with the UK in November. We are also prepared for a no-deal scenario but of course we are best prepared for a no-Brexit scenario."""

894 :非科技

"Congrats, Sarah! Honored to work with such a talented writer (and friend and cat-sitter)https://twitter.com/AAASKavli/status/1062756826939305984 …"

895 

933 :非科技

"Link found between breastfeeding and autism but more research needed #bfconf http://www.skepticalob.com/2018/11/link-found-between-breastfeeding-and-autism-but-more-research-needed.html …pic.twitter.com/pXccc7rM3B"

934 :非科技

"Idaho National Laboratory‘s Dave Petti has been all over the world discussing a @MIT report he led on the future of #nuclear. See where he is headed next!  http://energy.mit.edu/research/future-nuclear-energy-carbon-constrained-world/ …pic.twitter.com/vhrWx8t91a"

935 :非科技

"We have a packed agenda planned for #dpdksummit North America, Dec. 3-4 in San Jose! Join us to discuss DPDK roadmap, container networking, P4, hardware accelerators + more!https://events.linuxfoundation.org/events/dpdknorthamerica2018/ …"

936 :非科技

"Automation Anywhere raises $300 million from SoftBank’s Vision Fundhttps://ift.tt/2QJDuL2 "

937 :科技

"Excitement increases as @NASAVoyager 2 shows evidence it is close to entering interstellar space: https://go.nasa.gov/2DlU6om pic.t

975 :非科技

"Carbon breakdown in the rumen illuminated as multi-omics meets metagenomics and viromics  https://go.nature.com/2zTlWF9  #NBThighlight @NatureMicrobiol @kcwrighton"

976 :科技

".@BillyIdol becomes a US citizen: ""It‘s a nice day for a naturalization ceremony"" http://cos.lv/lA3i30mDcVo pic.twitter.com/pEVOrAgI3S"

977 :非科技

"Energy, Tariffs & the New NAFTA: What It Means for Idaho.  @wacleantech event in Boise on December 3.  http://ow.ly/y5EE30mCGqb  #USMCA"

978 :非科技

"A real privilege to have engineering superstar Mahika join us at the X Lab @WSUKLIVE today 7 year-old Mahika spoke to students about her invention, a walking stick that helps people with visual impairments which has been built by @UCLEngineering #inspireanengineer #SkillsLIVEpic.twitter.com/hyuhT9evZ1"

979 :非科技

".@Azure HPC leader @JasonAStowe joins the Intel Chip Chat podcast to discuss delivering HPC services atop Microsoft Azure cloud infrastructure. Tune in: https://intel.ly/2Ps76Qx  #SC18 #IntelHPCpic.

In [60]:
def label_file(path,i,j):
    with open(path,'r') as f:
        t_count = 0
        count=0
        for line in f:
            count += 1
            if count < i : continue   
            if count > j : break
            print(line)
            line = clean_url_replace_hashtag(line)
            line = sentences_to_indices(np.array([line]), w_to_ix, MAX_LEN)
            ans = model.predict(line)[0]
           
            if ans[0]>ans[1]:
                print(count,":非科技\n")
            else:
                print(count,":科技\n")
                t_count += 1
            
        print(t_count/(j-i+1) * 100,'%')    
            

In [22]:
def load_data(path):
    with open(path, 'r', encoding = 'utf8') as f:
        buffer = f.readlines()
    return buffer

In [38]:
def sentences_to_indices(X, word_to_index, max_len,debug=False):
    """
    a(described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = nlp(X[i].lower())
        
        # Initialize j to 0
        
        if len(sentence_words)> max_len:
            continue
        # Loop over the words of sentence_words
        else:
            j = 0
            for w in sentence_words:

                try:
                    X_indices[i, j] = word_to_index[str(w)]
                    j += 1
                except KeyError:
                    continue
        if debug:print("已完成:{}/{}".format(i+1,m),end='\r')
                

    return X_indices

def indices_to_sentence(X, index_to_word):
    sentence = ""
    for i in X :
        if i!=0:
            try:
                sentence += (index_to_word[int(i)] + ' ')
            except KeyError:
                sentence += 'ERR '
    return sentence

In [64]:
# save dict
np.save("word_to_index.npy",w_to_ix)
np.save("index_to_word.npy",ix_to_w)
